# Installing Transformer and imports

In [ ]:
!export http_proxy="http://proxy21.iitd.ac.in:3128"
!export https_proxy="http://proxy21.iitd.ac.in:3128"

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import math
import tensorflow as tf
import seaborn as sns
# import tensorflow_datasets as tfds
from bs4 import BeautifulSoup
from sklearn import metrics as sk_metrics
import nltk
import re
from sklearn.utils import shuffle
import tqdm as notebook_tqdm

# Data read and split...for some accuracy numbers

In [ ]:
col_list = ["id", "tweet", "label"]
our_data = pd.read_csv("/home/shivangib/IRMIDIS-2022/2022/vax_train.csv",usecols=col_list)
df_test = pd.read_csv("/home/shivangib/IRMIDIS-2022/2022_test/vax_test.csv")
y_map = {
    'ProVax': 0, 
    'AntiVax': 1, 
    'Neutral': 2
}

our_data['Sentiment_Cat'] = our_data['label'].map(y_map)

In [ ]:
our_data

In [ ]:
df_test

Uncomment if want testing results

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
df = our_data
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_valid_index in split.split(df, df.Sentiment_Cat):
    train_set = df.iloc[train_index]
    test_valid_set = df.iloc[test_valid_index]

# split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
# for test_index, valid_index in split2.split(test_valid_set, test_valid_set.Sentiment_Cat):
#     test_set = test_valid_set.iloc[test_index]
#     valid_set = test_valid_set.iloc[valid_index]

print(train_set.shape)
print(test_valid_set.shape)
# print(test_set.shape)
# print(valid_set.shape)

df_train = train_set
# df_test = test_set
df_valid = test_valid_set

In [ ]:
df_train.info()

In [ ]:
df_train.Sentiment_Cat.value_counts()

In [ ]:
df_test.Sentiment_Cat.value_counts()

In [ ]:
df_train.head()

In [ ]:
df_train['tweet'].iloc[0]

In [ ]:
df_test.head()

In [ ]:
# y_map = {
#     'ProVax': 0, 
#     'AntiVax': 1, 
#     'Neutral': 2
# }

# df_train['Sentiment_Cat'] = df_train['label'].map(y_map)
# df_test['Sentiment_Cat'] = df_test['label'].map(y_map)

# Pre-processing

In [ ]:
import sys
import re
# !pip install emoji --quiet
import emoji
# !pip install contractions --quiet
import contractions
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
import unicodedata

In [ ]:
def cleaning(text):
  text= text.lower()
  text= emoji.demojize(text)
  text=contractions.fix(text)
  text=text.strip()
  text=text.replace('[^\w\s]','')
  text=re.sub(r'http\S+', '', text)
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
  text = REPLACE_BY_SPACE_RE.sub(' ' , text)
  text = BAD_SYMBOLS_RE.sub(' ',text)
  
  return text
df_train['tweet_cleaned'] = df_train['tweet'].apply(cleaning)
df_test['tweet_cleaned'] = df_test['tweet'].apply(cleaning)
df_valid['tweet_cleaned'] = df_valid['tweet'].apply(cleaning)
df_train['tweet_cleaned'].iloc[0]

In [ ]:
x_train, y_train = df_train['tweet_cleaned'].values, np.array(df_train['Sentiment_Cat'], dtype=np.int64)
# x_test, y_test = df_test['tweet_cleaned'].values, np.array(df_test['Sentiment_Cat'], dtype=np.int64)
x_valid, y_valid = df_valid['tweet_cleaned'].values, np.array(df_valid['Sentiment_Cat'], dtype=np.int64)


x_test = df_test['tweet_cleaned'].values

# Data Exploration

In [ ]:
data = df_train
Neutral_data = data[data['Sentiment_Cat'] == 2]
Antivax_data = data[data['Sentiment_Cat'] == 1]
Provax_data = data[data['Sentiment_Cat'] == 0]

In [ ]:
# df = Antivax_data
a=[]
for i in range(len(Antivax_data)):
  a.append(len(Antivax_data['tweet_cleaned'].iloc[i].split()))
print(max(a))
# df = Provax_data
b=[]
for i in range(len(Provax_data)):
  b.append(len(Provax_data['tweet_cleaned'].iloc[i].split()))
print(max(b))
# df = Neutral_data
c=[]
for i in range(len(Neutral_data)):
  c.append(len(Neutral_data['tweet_cleaned'].iloc[i].split()))
print(max(c))
print(len(Neutral_data), len(Antivax_data), len(Provax_data))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
plt.hist(a,bins= 80, label = "AntiVax")
plt.legend()
plt.xlabel('Length')
plt.ylabel('Frequecy')
plt.title('Length Histogram')
plt.show()
plt.hist(b,bins= 60, label = "ProVax")
plt.legend()
plt.xlabel('Length')
plt.ylabel('Frequecy')
plt.title('Length Histogram')
plt.show()
plt.hist(c, bins= 60, label = "Neutral")
plt.legend()
plt.xlabel('Length')
plt.ylabel('Frequecy')
plt.title('Length Histogram')
plt.show()

# BERT

In [ ]:
from tokenizers import BertWordPieceTokenizer
import transformers

In [ ]:
# First load the real tokenizer
# tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased', lower=True)

# Save the loaded tokenizer locally
# tokenizer.save_pretrained('.')

tokenizer = transformers.BertTokenizerFast.from_pretrained('.')

# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=200):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
max_len=70
x_train_enc = fast_encode(x_train, fast_tokenizer, maxlen=max_len)
x_valid_enc = fast_encode(x_valid, fast_tokenizer, maxlen=max_len)
x_test_enc = fast_encode(x_test, fast_tokenizer, maxlen=max_len)

In [ ]:
NUM_CLASSES = df_train['Sentiment_Cat'].nunique()

# bert_transformer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
# bert_transformer = transformers.TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_CLASSES)
bert_transformer = transformers.TFBertForSequenceClassification.from_pretrained('digitalepidemiologylab/covid-twitter-bert-v2', num_labels=NUM_CLASSES)
# bert_transformer.save_pretrainined(".")

input = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
sequence_output = bert_transformer(input)[0]
# cls_token = sequence_output[:, 0, :]
# output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(cls_token)

model = tf.keras.Model(inputs=input, outputs=sequence_output)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# call backs
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

history = model.fit(
    x_train_enc,
    y_train,
    batch_size=16,
    validation_data=(x_valid_enc, y_valid),
    epochs=50,
    callbacks=[early_stop_callback])

# Prediction 

In [ ]:
ypred = model.predict(x_test_enc)
ypred
# df_test['ypred'] = np.argmax(ypred, axis=1)
# df_test['ypred_cat'] = np.where(df_test['ypred']>=0.5, 1, 0)
# df_test['ypred_cat'] = np.argmax(ypred, axis=1)
# df_test

In [ ]:
print(np.argmax(ypred, axis=1))

In [ ]:
df_test['ypred'] = np.argmax(ypred, axis=1)
df_test

In [ ]:
# df_test1["id"]= df_test["id"]
df_test.to_csv("submission.csv")

In [ ]:
y_map_inv = {
    0: 'ProVax',
    1: 'AntiVax',
    2: 'Neutral'
}

# y_map = {
#     'ProVax': 0, 
#     'AntiVax': 1, 
#     'Neutral': 2
# }

sns.heatmap(
    pd.crosstab(df_test['ypred'].map(y_map_inv), df_test['Sentiment_Cat'].map(y_map_inv)),
    annot=True,
    fmt='0.0f',
    cbar=False,
    linewidth=2,
    cmap='coolwarm'     
)

plt.xlabel('True')
plt.ylabel('Predicted')
plt.title('Confusion Matrix');

In [ ]:
y_test = df_test['Sentiment_Cat'].map(y_map_inv)
y_pred = df_test['ypred'].map(y_map_inv)
#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Class 1', 'Class 2', 'Class 3']))


In [7]:
y_map_inv = {
    0: 'ProVax',
    1: 'AntiVax',
    2: 'Neutral'
}

In [5]:
import pandas as pd
df1 = pd.read_csv("task1_run1.csv")
df2 = pd.read_csv("task1_run2.csv")

In [11]:
df2.head(10)

,id,pred
0,1472958183882719233t,ProVax
1,1449995880904470529t,AntiVax
2,1480413964265660416t,Neutral
3,1408119497962057742t,ProVax
4,1439361704320806915t,Neutral
5,1478769637282230274t,ProVax
6,1472297934633017344t,ProVax
7,1520114105985769475t,ProVax
8,1423827464573255683t,Neutral
9,1479557209751175174t,Neutral


In [8]:
df1['pred'] = df1['pred'].map(y_map_inv)

In [10]:
df2['pred'] = df2['pred'].map(y_map_inv)

In [12]:
df1.pred.value_counts()

Neutral    341
ProVax     248
AntiVax    128
Name: pred, dtype: int64

In [13]:
df2.pred.value_counts()

Neutral    296
ProVax     220
AntiVax    201
Name: pred, dtype: int64

In [14]:
df1.to_csv("task1_run1.csv")

In [15]:
df2.to_csv("task1_run2.csv")